<a href="https://colab.research.google.com/github/MihaiDogariu/Keysight-Deep-Learning-Fundamentals--v2-/blob/main/scripts/Unit_1_Intro_on_tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to tensors


This notebook makes a short introduction to working with *tensors*, the fundamental object of Deep Learning frameworks, such as PyTorch or Tensorflow.

Generally speaking, tensors are N-dimensional arrays, and can be considered an extension of classical NumPy arrays. The core difference between the two, is that tensors are specifically designed to be run on GPUs, making tensor operations a few orders of magnitude faster than their CPU counter-parts.

In [ ]:
!pip install ipython-autotime
import torch
import numpy as np

# We will use the autotime command to get the running time of each code block and investigate the difference
%load_ext autotime

time: 0 ns (started: 2025-03-27 01:38:42 +02:00)


First, lets create two similar N-D arrays: an np array and a tensor, of the same dimensions. We will populate them with:
- [`np.random.rand()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html)
- [`torch.rand()`](https://pytorch.org/docs/stable/generated/torch.rand.html)

By default, np arrays use `float64`, whereas tensors use `float32` data. We can convert the np arrays to a similar data type and re-run the same operations. We can use the `np.float32()` cast operator or the [`np.ndarray.astype()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) function.


In [ ]:
array_size = (3, 3)

# 2D arrays
array1 = np.random.rand(*array_size).astype(np.float32)
array2 = np.random.rand(*array_size).astype(np.float32)

# 2D tensors
tensor1 = torch.rand(*array_size)
tensor2 = torch.rand(*array_size)

time: 15 ms (started: 2025-03-27 01:38:42 +02:00)


Let's print the data. We can also have a look at the type of data (`.dtype` attribute).

Tensors, unlike np arrays, can be run on both CPU and GPU. They have an attribute, `.device` that holds this particular information. Let's print and see where the tensor resides upon default creation.

In [ ]:
print('Np array values:')
print(array1)
print('\nTensor values:')
print(tensor1)

print(f'\nNp array data type is {array1.dtype}')
print(f'Tensor data type is {tensor1.dtype}')
print(f'Tensor is loaded on {tensor1.device}')

Np array values:
[[0.09254928 0.19802298 0.6852597 ]
 [0.8152934  0.6896014  0.3033488 ]
 [0.38212073 0.5211278  0.59452564]]

Tensor values:
tensor([[0.2452, 0.4294, 0.7648],
        [0.0185, 0.3601, 0.2743],
        [0.1359, 0.5836, 0.6405]])

Np array data type is float32
Tensor data type is torch.float32
Tensor is loaded on cpu
time: 16 ms (started: 2025-03-27 01:38:42 +02:00)


Now let's see how fast the operations are on both types of arrays.

First, run a multiplication operation on the np arrays with [`np.matmul()`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html#numpy-matmul):

In [ ]:
res_np = np.matmul(array1, array2)

time: 0 ns (started: 2025-03-27 01:38:43 +02:00)


Then, a similar multiplication on the tensors with [`torch.matmul()`](https://pytorch.org/docs/stable/generated/torch.matmul.html#torch.matmul):

In [ ]:
res_tensor = torch.matmul(tensor1, tensor2)

time: 15 ms (started: 2025-03-27 01:38:43 +02:00)


The processing time is nearly identical in this case. So let's see the GPU speedup (if any). In order to do so, we must first check if our system has a dedicated GPU with CUDA support. We can check with `torch.cuda.is_available()`.

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cuda
time: 1.72 s (started: 2025-03-27 01:38:43 +02:00)


If there is such a device available, then we can load the tensors on it, with the [`torch.Tensor.to()`](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html) method.

In [ ]:
tensor1 = tensor1.to(device)
tensor2 = tensor2.to(device)

time: 109 ms (started: 2025-03-27 01:38:44 +02:00)


And then we re-run the multiplication, only that this time it will be done on the GPU, instead of the CPU.

In [ ]:
res_tensor = torch.matmul(tensor1, tensor2)

time: 109 ms (started: 2025-03-27 01:38:45 +02:00)


Well, the difference is not spectacular at all, but that is due to the very small size of the arrays. We have to pump those numbers up! Those are rookie numbers.

So we will re-run the above operations, but on a larger scale. Switch `array_size` to 100 x 100 x 100 x 100 and see what the GPU does now.

In [ ]:
array_size = (100, 100, 100, 100)
array1 = np.float32(np.random.rand(*array_size))
array2 = np.float32(np.random.rand(*array_size))
tensor1 = torch.rand(*array_size).to(device)
tensor2 = torch.rand(*array_size).to(device)

time: 4 s (started: 2025-03-27 01:38:45 +02:00)


In [ ]:
res_np = np.matmul(array1, array2)

time: 640 ms (started: 2025-03-27 01:38:49 +02:00)


In [ ]:
res_tensor = torch.matmul(tensor1, tensor2)

time: 16 ms (started: 2025-03-27 01:38:50 +02:00)


In order to extract numeric values from the tensor and use them as Python numbers, we have several options:
- index the tensor and then call [`torch.Tensor.item()`](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html) on a single value;
- convert the tensor to a list [`torch.Tensor.tolist()`](https://pytorch.org/docs/stable/generated/torch.Tensor.tolist.html#torch.Tensor.tolist) and treat it as a Python list;
- convert the tensor to a np array [`torch.Tensor.numpy()`]() and treat it as a np array - this step needs the tensor to be transfered back on CPU before-hand with: `torch.Tensor.cpu()`.

In [ ]:
x = res_tensor.cpu().numpy()
print(x[0][0])

[[21.486055 22.8759   25.30208  ... 22.386066 20.962614 24.18737 ]
 [25.731264 24.931953 27.061218 ... 25.293928 25.31982  28.41498 ]
 [26.403093 25.883512 24.406162 ... 24.500051 24.762085 27.099932]
 ...
 [25.51644  24.542372 28.432758 ... 26.357706 24.964693 27.648375]
 [24.225964 25.506601 25.464682 ... 26.73598  24.753082 27.741743]
 [30.166197 27.59787  27.955866 ... 30.11703  29.02287  29.481533]]
time: 125 ms (started: 2025-03-27 01:38:50 +02:00)
